# 1.0 Setup

Instalação das dependencias nescesarias:

In [ ]:
# Instala a biblioteca gradio, que permite criar interfaces de usuário para protótipos de modelos de aprendizado de máquina.
!pip install gradio

# Instala a biblioteca sentence-transformers, usada para sentence embedding
!pip install sentence-transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.7/19.7 MB 70.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 51.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.0/57.0 kB 6.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 288.4/288.4 kB 28.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.4/75.4 kB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 27.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.5/50.5 kB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.0/137.0 kB 11.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.7/45.7 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 k

In [ ]:
import locale
locale.getpreferredencoding = lambda: 'UTF-8'

Instalação das bibliotecas utilizadas:

In [ ]:
# Importa o módulo pickle, que permite serializar e desserializar objetos Python.
import pickle

# Importa o pandas, uma biblioteca de manipulação de dados popular.
import pandas as pd

# Importa o numpy, uma biblioteca para operações matemáticas e de matriz.
import numpy as np

# Importa o SentenceTransformer da biblioteca sentence-transformers. Isso será usado para converter sentenças em vetores.
from sentence_transformers import SentenceTransformer

# Importa o módulo Tokenizer da biblioteca Keras. Este módulo é usado para converter sentenças em sequências de números.
from keras.preprocessing.text import Tokenizer

# Importa a biblioteca gradio, que será usada para criar a interface do usuário.
import gradio as gr

# Importa a biblioteca matplotlib.pyplot, usada para a criação de gráficos e visualizações.
import matplotlib.pyplot as plt

# Instalação das dependencias PLN
!pip install unidecode
!pip install -U spacy
!python -m spacy download pt_core_news_sm

#Bibliotecas de PLN:

from unidecode import unidecode
import re
import spacy
from spacy.lang.pt.stop_words import STOP_WORDS
nlp = spacy.load('pt_core_news_sm')

#Biblioteca nuvem de palavras:
from wordcloud import WordCloud
import tempfile

# 2.0 Deploy do modelo

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#importando o modelo
modeloTransformersImportado = ('/content/drive/MyDrive/Modulo_6_grupo_1/modelo_rede_neural_Transformers.pkl')

In [ ]:
#importando o modelo
modeloTransformersImportado = ('/content/modelo_rede_neural_Transformers.pkl')

In [ ]:
with open(modeloTransformersImportado, 'rb') as f:
    modeloTransformers = pickle.load(f)

In [ ]:
#Carregando o vetorizador
vetorizador = SentenceTransformer('distiluse-base-multilingual-cased')

# 3.0 Teste do modelo importado

In [ ]:
#Criação de dados para teste
dadosteste = {
    'frase': ["esse o pior banco que ja vi em toda minha vida","muito ruim", "muito bom", "adorei", "nao gostei muito nao", "."],
}
dfteste = pd.DataFrame(dadosteste)
dfteste

,frase
0,esse o pior banco que ja vi em toda minha vida
1,muito ruim
2,muito bom
3,adorei
4,nao gostei muito nao
5,.


In [ ]:
# função para prever os sentimentos
def prever1(dados):
  dados = vetorizador.encode(dados.tolist())
  dados = pd.DataFrame(dados)
  previsoes = modeloTransformers.predict(dados)
  previsoes = np.argmax(previsoes, axis=1)
  return previsoes

In [ ]:
# teste da função
prever1(dfteste["frase"])

1/1 [==============================] - 0s 227ms/step


array([2, 2, 0, 0, 0, 1])

In [ ]:
# guardando os resultados
dfteste['sentimento'] = pd.Series(prever1(dfteste["frase"]))

1/1 [==============================] - 0s 25ms/step


Abaixo e possivel visualizar os resultados obtidos no teste:

In [ ]:
# Vizualização dos resultados obtidos
dfteste

,frase,sentimento
0,esse o pior banco que ja vi em toda minha vida,2
1,muito ruim,2
2,muito bom,0
3,adorei,0
4,nao gostei muito nao,0
5,.,1


# 4.0 Pipeline

Vale mencionar que os tratamentos definidos aqui estão documentados e registrados no notebook de treinamento, eles estão aqui apenas para funcionamento do dashboard

In [ ]:
#Código para remoção dos acentos
def remocaoAcentos(dataframe):
    dataframe = dataframe.applymap(lambda x: unidecode(str(x)))
    return dataframe

In [ ]:
def tratamentoMaiusculas(dataframe):
    # Pré-processamento das letras maiúsculas
    dataframe = dataframe.applymap(lambda x: x.lower() if isinstance(x, str) else x)
    # Exibição dos dados pré-processados
    return dataframe

In [ ]:
def tratamentoAbreviacoes(coluna):
    coluna = coluna.astype(str)
    coluna = coluna.str.lower()

    # Dicionário com as abreviações e suas expansões, incluindo as palavras a serem substituídas
    abreviacoes = {
        r'\bvc\b': 'você',
        r'\btbm\b': 'também',
        r'\bpq\b': 'porque',
        r'\bmt\b': 'muito',
        r'\bmto\b': 'muito',
        r'\bblz\b': 'beleza',
        r'\bjah\b': 'já',
        r'\bbj\b': 'beijo',
        r'\bflw\b': 'falou',
        r'\bvlw\b': 'valeu',
        r'\bkrl\b': 'caralho',
        r'\bqnd\b': 'quando',
        r'\bfds\b': 'fim de semana',
        r'\bqq\b': 'qualquer',
        r'\bmsg\b': 'mensagem',
        r'\bpra\b': 'para',
        r'\bp\b': 'para',
        r'\bpqp\b': 'puta que pariu',
        r'\bvlh\b': 'velho',
        r'\bnss\b': 'nossa',
        r'\bngm\b': 'ninguém',
        r'\bqm\b': 'quem',
        r'\bobs\b': 'observação',
        r'\bqt\b': 'quanto',
        r'\btbm ñ\b': 'também não',
        r'\bpf\b': 'por favor',
        r'\bpdc\b': 'pode crer',
        r'\btalkei\b': 'ok',
        r'\bd+\b': 'demais',
        r'\bdms\b': 'demais',
        r'\bsqn\b': 'só que não',
        r'\bmsm\b': 'mesmo',
        r'\bn/ao\b': 'não',
        r'\bn/\b': 'não',
        r'\bblz\b': 'beleza',
        r'\bq\b': 'que',
        r'\btd\b': 'tudo',
        r'\bpfvr\b': 'por favor',
        r'\bgnt\b': 'gente',
        r'\bto\b': 'estou',
        r'\bvcs\b': 'vocês',
        r'\bn/oa\b': 'não',
        r'\bsla\b': 'sei lá',
        r'\bflws\b': 'falou',
        r'\bbjos\b': 'beijos',
        r'\bkd\b': 'cadê',
        r'\blgl\b': 'legal',
        r'\bnum\b': 'número',
        r'\bjá q\b': 'já que',
        r'\bmano\b': 'irmão',
        r'\bpke\b': 'porque',
        r'\bpr\b': 'para',
        r'\btpm\b': 'tensão pré-menstrual',
        r'\btá\b': 'está',
        r'\bqmto\b': 'muito',
        r'\bñ\b': 'não',
        r'\bvamu\b': 'vamos',
        r'\btmj\b': 'estamos juntos',
        r'[0-9]': '',
        r'@\w+': '',
        r'[^\w\s]': '',
        r'\bwww\.[^\s]*': '',
        r'\bola\b': '',
        r'\n': '',
        r'\bbtg\b': '',
        r'\bpactual\b': '',
        r'\bvocê\b': '',
        r'\bdia\b': '',
        r'\bjá\b': '',
        r'\bsão\b': '',
        r'\bsó\b': '',
        r'\blink\b': '',
        r'\br\b': '',
        r'\brepost\b': '',
        r'\baté\b': '',
        r'\bpaulo\b': '',
        r'\bbio\b': '',
        r'\bdia\b': '',
        r'\bdias\b': '',
        r'\bsera\b': '',
        r'\blá\b': '',
        r'\bmês\b': '',
        r'\bestão\b': '',
        r'\balém\b': '',
        r'\bano\b': '',
        r'\banos\b': '',
    }

    # Aplica a substituição de cada abreviação no dataframe
    for abreviacao, expansao in abreviacoes.items():
        coluna = coluna.apply(lambda x: re.sub(abreviacao, expansao, x, flags=re.IGNORECASE))

    return coluna

In [ ]:
def remocaoStopWords(coluna):
    listaSemStopWords = []
    for texto in coluna:
        # converte o texto em um documento do Spacy
        doc = nlp(texto)
        # filtra os tokens que não são stop words
        tokenSemStopwords = [token.text for token in doc if not token.is_stop]
        # adiciona a lista de frases sem stop words
        listaSemStopWords.append(' '.join(tokenSemStopwords))
    # substitui a coluna original pelos textos sem stop words
    coluna[:] = listaSemStopWords
    return coluna

In [ ]:
def pipeline(dados):
  #cria um novo df a ser tratado
  dadosT =  pd.DataFrame({'dadosTratados': dados})
  # substituição das letras maiusculas dos dados
  dadosT['dadosTratados'] = tratamentoMaiusculas(dadosT)
  #tratamento de stopwords
  dadosT['dadosTratados'] = remocaoStopWords(dadosT['dadosTratados'])
  #tratamento de abreviações
  dadosT['dadosTratados'] = tratamentoAbreviacoes(dadosT['dadosTratados'])
  # remove os acentos dos dados
  dadosT['dadosTratados'] = remocaoAcentos(dadosT)
  return dadosT['dadosTratados']

# 5.0 Contagem de palavras

In [ ]:
#Definição de função para criação de dicionario
def tokenizacaoPalavra(comentarios):
  tokenizer = Tokenizer()
  tokenizer.fit_on_texts(comentarios)
  return tokenizer

In [ ]:
#Definição de função para contagem de palavras
def contarPalavrasFrequentes(tokenizer, top_n):
    contagem_palavras = tokenizer.word_counts
    palavras_frequentes = dict(sorted(contagem_palavras.items(), key=lambda item: item[1], reverse=True)[:top_n])
    return palavras_frequentes

Teste

In [ ]:
#Teste das funções em conjunto
tokenizer = tokenizacaoPalavra(dfteste["frase"])
palavras_frequentes = contarPalavrasFrequentes(tokenizer, 15)
print(palavras_frequentes)

{'muito': 3, 'nao': 2, 'esse': 1, 'o': 1, 'pior': 1, 'banco': 1, 'que': 1, 'ja': 1, 'vi': 1, 'em': 1, 'toda': 1, 'minha': 1, 'vida': 1, 'ruim': 1, 'bom': 1}


In [ ]:
# Função para calcular as palavras mais frequentes
def frequentes(dados):
  dic = tokenizacaoPalavra(dados)
  palavras_frequentes = contarPalavrasFrequentes(dic, 15)

  return palavras_frequentes

Teste

In [ ]:
# teste da função
frequentes(dfteste["frase"])

{'muito': 3,
 'nao': 2,
 'esse': 1,
 'o': 1,
 'pior': 1,
 'banco': 1,
 'que': 1,
 'ja': 1,
 'vi': 1,
 'em': 1,
 'toda': 1,
 'minha': 1,
 'vida': 1,
 'ruim': 1,
 'bom': 1}

## 5.1 Grafico com palavras mais frequentes

In [ ]:
# definição de função para gerar uma grafico com as palavras mais frequentes
def gerarGraficoFrequencia(frequencia):
  # Cria um DataFrame a partir do dicionário de frequência
  df_frequencia = pd.DataFrame.from_dict(frequencia, orient='index', columns=['Contagem'])

  # Ordena o DataFrame pela contagem em ordem decrescente
  df_frequencia = pd.DataFrame.from_dict(frequencia, orient='index', columns=['Contagem'])

  # Ordena o DataFrame pela contagem em ordem decrescente
  df_frequencia = df_frequencia.sort_values('Contagem', ascending=False)

  # Cria o gráfico de barras
  plt.figure(figsize=(10, 6))
  plt.bar(df_frequencia.index, df_frequencia['Contagem'])
  plt.xlabel('Palavra')
  plt.ylabel('Contagem')
  plt.title('Frequência das Palavras')
  plt.xticks(rotation=45)
  plt.tight_layout()
  # Armazena o gráfico de barras na variável graficofrequencia
  graficofrequencia = plt.gcf()
  return graficofrequencia

# 6.0 Sentimentos frequentes

In [ ]:
#função para definir o sentimento gerado pelas palavras mais frequentes
def sentimentoFrequentes(dados):
  sentimentoMaisFrequentes = frequentes(dados)
  sentimentoMaisFrequentes = list(sentimentoMaisFrequentes.keys())
  sentimentoMaisFrequentes = pd.DataFrame({'palavra': sentimentoMaisFrequentes})
  sentimentoM = prever1(sentimentoMaisFrequentes['palavra'])
  sentimentoMaisFrequentes['previsto'] = sentimentoM
  return sentimentoMaisFrequentes

In [ ]:
sentimentoFrequentes(dfteste["frase"])

1/1 [==============================] - 0s 39ms/step


,palavra,previsto
0,muito,0
1,nao,1
2,esse,1
3,o,1
4,pior,1
5,banco,1
6,que,1
7,ja,1
8,vi,1
9,em,1


# 7.0 Nuvem de palavras

In [82]:
def gerarNuvemPalavras(dados):

  dic = tokenizacaoPalavra(dados)
  dicionarioFreq = contarPalavrasFrequentes(dic, 200)
  # Cria uma instância da classe WordCloud com as configurações desejadas
  nuvemPalavras = WordCloud(width=6400, height=3200, background_color='white', colormap='viridis')

  # Gera a nuvem de palavras a partir do dicionário de frequências
  nuvemPalavras.generate_from_frequencies(dicionarioFreq)

  # Retorna a nuvem de palavras gerada
  return nuvemPalavras

In [83]:
def exibirNuvemPalavras(dicionarioFreq):
    nuvem = gerarNuvemPalavras(dicionarioFreq)

    # Salva a nuvem de palavras em um arquivo temporário
    with tempfile.NamedTemporaryFile(suffix='.png', delete=False) as temp_file:
        arquivo_temporario = temp_file.name
        nuvem.to_file(temp_file.name)

    return arquivo_temporario

In [ ]:
# Teste
exibirNuvemPalavras(dfteste["frase"])

# 8.0 Grafico Sentimento - Mês

In [ ]:
#Definindo função para gerar um grafico de sentimentos por tempo
def prepare_and_plot(dfData, _nome_da_coluna_datas):
    # Preparação dos dados
    dfData[_nome_da_coluna_datas] = dfData[_nome_da_coluna_datas].str.slice(1, 11)
    dfData[_nome_da_coluna_datas] = pd.to_datetime(dfData[_nome_da_coluna_datas])
    dfData['ano'] = dfData[_nome_da_coluna_datas].dt.year
    dfData['mes'] = dfData[_nome_da_coluna_datas].dt.month

    # Filtragem dos sentimentos
    df_negativos = dfData[dfData['sentimento'] == "NEGATIVE"]
    df_positivos = dfData[dfData['sentimento'] == "POSITIVE"]
    df_neutros = dfData[dfData['sentimento'] == "NEUTRAL"]

    # Contagem dos registros
    contagem_negativos_por_mes_ano = df_negativos.groupby(['ano','mes']).size().reset_index(name='quantidade_negativos')
    contagem_positivos_por_mes_ano = df_positivos.groupby(['ano','mes']).size().reset_index(name='quantidade_positivos')
    contagem_neutros_por_mes_ano = df_neutros.groupby(['ano','mes']).size().reset_index(name='quantidade_neutros')

    # Concatenação do ano e mês
    contagem_negativos_por_mes_ano['ano_mes'] = contagem_negativos_por_mes_ano['ano'].astype(str) + '-' + contagem_negativos_por_mes_ano['mes'].astype(str).str.zfill(2)
    contagem_positivos_por_mes_ano['ano_mes'] = contagem_positivos_por_mes_ano['ano'].astype(str) + '-' + contagem_positivos_por_mes_ano['mes'].astype(str).str.zfill(2)
    contagem_neutros_por_mes_ano['ano_mes'] = contagem_neutros_por_mes_ano['ano'].astype(str) + '-' + contagem_neutros_por_mes_ano['mes'].astype(str).str.zfill(2)

    # Geração do gráfico
    fig, ax = plt.subplots(figsize=(15,7))
    fig.patch.set_facecolor('#1F2937') #mudar a cor de fundo do grafico

    ax.plot(contagem_neutros_por_mes_ano['ano_mes'], contagem_neutros_por_mes_ano['quantidade_neutros'], linestyle='-', label='Neutro')
    ax.plot(contagem_negativos_por_mes_ano['ano_mes'], contagem_negativos_por_mes_ano['quantidade_negativos'], linestyle='-', label='Negativo')
    ax.plot(contagem_positivos_por_mes_ano['ano_mes'], contagem_positivos_por_mes_ano['quantidade_positivos'], linestyle='-', label='Positivo')

    ax.set_xlabel('Ano-Mes', color='white')
    ax.set_ylabel('Quantidade', color='white')
    ax.set_title('Gráfico de sentimentos em função do tempo', color='white')
    plt.xticks(rotation=90, color='white')
    plt.yticks(color='white')
    ax.legend()
    ax.grid(True)

    return fig

# 9.0 Gradio


In [ ]:
# Criação de dashBoard referente a analize de sentimentos, para ser exibido no gradio
def sentimento_dash(file, nome_da_coluna_texto, nome_da_coluna_datas):
    df = pd.read_csv(file.name)
    df_frequencia = df
    nome_da_coluna = nome_da_coluna_texto


    df['sentimento'] = pd.Series(prever1(df[nome_da_coluna_texto]))
    df['sentimento'] = df['sentimento'].replace({0: 'POSITIVE',1: 'NEUTRAL', 2: 'NEGATIVE'})

    #lista dos textos
    df['Primeiras 5 frases'] = df[nome_da_coluna_texto] + '  -  [' + df['sentimento'] + ']'
    texto = pd.DataFrame(df['Primeiras 5 frases']).head()



    #criacao do grafico
    quantidades = df['sentimento'].value_counts()
    labels = quantidades.index
    fig, ax = plt.subplots()
    fig.patch.set_facecolor('#FFFF') #mudar a cor de fundo do grafico

    ax.pie(quantidades, labels=labels, autopct='%1.1f%%')
    ax.set_title('Sentimentos do corpus')

    x = str(quantidades)

    #tamanho do corpus do corpus
    num_tuplas = "Número de Tuplas: " + str(len(df))


    fig2 = prepare_and_plot(df, nome_da_coluna_datas)


    return [num_tuplas,x, fig,fig2]

In [ ]:
# Criação dos dados do dashboard referente à analize das palavras mais frequentes
def frequencia_dash(file, nome_da_coluna_texto):
    df = pd.read_csv(file.name)

    #frequencia
    frequencia = frequentes(pipeline(df[nome_da_coluna_texto]))

    # Armazena o gráfico de barras na variável graficofrequencia
    graficofrequencia = gerarGraficoFrequencia(frequencia)

    nuvemParaPlotar = exibirNuvemPalavras(pipeline(df[nome_da_coluna_texto]))


    return graficofrequencia, nuvemParaPlotar

In [ ]:
#Iniciando gradio
with gr.Blocks() as iface1:
    with gr.Tab("Arquivo - Sentimento"):
        text_input = [gr.File(),gr.Textbox(),gr.Textbox()]
        text_button = gr.Button("Aplicar")
        text_output = [gr.Textbox(),gr.Textbox(),gr.Plot(),gr.Plot()]
    with gr.Tab("Arquivo - Frequencia"):
        image_input = [gr.File(),gr.Textbox()]
        image_button = gr.Button("Flip")
        image_output = [gr.Plot(), gr.Image()]

    text_button.click(sentimento_dash, inputs=text_input, outputs=text_output)
    image_button.click(frequencia_dash,inputs=image_input, outputs=image_output)

iface1.launch()